# RAG - LLM - Parsing - Chunking

Parsing is the process of extracting raw text from documents such as PDFs, .docx files, youtube videos and so on. It depends on the type of data you want to parse.

For this LLM, only pdfs will be parsed

## Load libraries

In [19]:
import pymupdf
import pymupdf4llm
import sys
from datetime import datetime
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

sys.path.append("..")

from gcp_utils.gcs import get_file
from rag_llm_energy_expert.config import GCP_CONFIG, LLM_CONFIG

In [ ]:


model = SentenceTransformer("all-MiniLM-L6-v2", trust_remote_code=True)
# In case you want to reduce the maximum length:
#model.max_seq_length = 8192

queries = [
    "how much protein should a female eat",
    "summit define",
]
documents = [
    "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
    "Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments.",
]

#query_embeddings = model.encode(queries, prompt_name="query")
document_embeddings = model.encode(documents)

#scores = (query_embeddings @ document_embeddings.T) * 100
print(document_embeddings.shape)

(2, 384)


## Initialize config classes

In [6]:
gcp_config = GCP_CONFIG()
llm_config = LLM_CONFIG()

## Parsing PDFs

There are tons of libraries to extract data from PDFs, nevertheless, [*PyMuPDF*](https://pypi.org/project/pymupdf4llm/) is one of the best libraries because:

- Detects standard text and tables
- Header lines are identified via de font size and appropiately prefixed with one or more '#' tags.
- Bold, italic, mono-spaced text and code blocks are detected and formatted accordingly.
- By default, all document pages are processed.
- Support for pages with multiple text columns.
- Support image or vector graphic on the page and they're stored as an image.
- ***Support for page chunks***. Instead of returning one large string for the whole document, a list of dictionaries can be generated. One for each page.

*All the data parsed here comes from GCP*

Reading into memory is faster than download the pdf into a file system, and then read the file from there. Moreover, it's useful when you do not have a persistent memory or you want to work directly with the file.

### Extracting data from PDF

In [13]:
file_to_read = "documents/summaries/resumen_reforma_energetica.pdf"
title = file_to_read.split("/")[-1].split(".")[0]

#Loads in memory a pdf stored in GCS
pdf_bytes = get_file(file_to_read)

# Create a Document object, it can be constructed from a file or from memory
# pymupdf.Document() method is exactly the same as pymupdf.open()
doc = pymupdf.Document(stream = pdf_bytes)

# Reads the PDF with its metadata and creates a list of dictionaries if chunking, or a string with all the content
md_text = pymupdf4llm.to_markdown(
        doc,
        # page_chunks = True, # Create a list of pages of the Document 
        # extract_words=True, # Adds key words to each page dictionary
        show_progress = False,
    )

md_text

'# Resumen Ejecutivo\n\n\n-----\n\n-----\n\n## I. Introducción\n\nLa Reforma Energética es un paso decidido rumbo a la modernización del sector energético de\nnuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamiento de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a\nnivel legistlación secundarias, surge del estudio y valoración de las distintas iniciativas presentadas por los partidos políticos representados en el Congreso.\n\n### La Reforma Energética tiene los siguientes objetivos y premisas fundamentales:\n\n1. Mantener la propiedad de la Nación sobre los hidrocarburos que se encuentran en el subsuelo.\n2. Modernizar y fortalecer, sin privatizar, a Petróleos Mexicanos (Pemex) y a la Comisión Federal de Electricidad (CFE) como Empresas Productivas del Estado, 100% públicas y 100%\nmexicanas.\n3. Reducir la exposición del país a los riesgos financieros, geológicos y ambientales en las actividades de e

### Chunking the md_text

In this case, we will use the text splitters from [langchain](https://python.langchain.com/api_reference/text_splitters/index.html). Mainly,we will be using the [Markdown](https://python.langchain.com/docs/how_to/markdown_header_metadata_splitter/) and the [RecursiveCharacterTextSplitter](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/) ones.

By default, MarkdownHeaderTextSplitter strips headers being split on from the output chunk's content. This can be disabled by setting: *strip_headers = False*, also, it strips white spaces and new lines. To preserve the original formatting of your Markdown documents, checkout [ExperimentalMarkdownSyntaxTextSplitter](https://python.langchain.com/api_reference/text_splitters/markdown/langchain_text_splitters.markdown.ExperimentalMarkdownSyntaxTextSplitter.html)

In [22]:
# Choose the headers to split on
headers_to_split_on=[("#", 'Header 1'), ("##", "Header 2"), ("###", "Header 3"), ("####", "Header 4"), ("#####", "Header 5")]

# Initialize a MarkdownHeaderTextSplitter object
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on, strip_headers=False)

# Split the text
md_header_splits = markdown_splitter.split_text(md_text)

print(len(md_header_splits))
md_header_splits

18


[Document(metadata={'Header 1': 'Resumen Ejecutivo'}, page_content='# Resumen Ejecutivo  \n-----  \n-----'),
 Document(metadata={'Header 1': 'Resumen Ejecutivo', 'Header 2': 'I. Introducción'}, page_content='## I. Introducción  \nLa Reforma Energética es un paso decidido rumbo a la modernización del sector energético de\nnuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamiento de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a\nnivel legistlación secundarias, surge del estudio y valoración de las distintas iniciativas presentadas por los partidos políticos representados en el Congreso.'),
 Document(metadata={'Header 1': 'Resumen Ejecutivo', 'Header 2': 'I. Introducción', 'Header 3': 'La Reforma Energética tiene los siguientes objetivos y premisas fundamentales:'}, page_content='### La Reforma Energética tiene los siguientes objetivos y premisas fundamentales:  \n1. Mantener la propiedad de la Nación sob

Once the data has been splitted into different chunks, we can split them more to adjust it to a specific chunk size and also specify the chunk overlap per each division already created. To do so, we can then apply any text splitter we want, such as RecursiveCharacterTextSplitter

- **Chunking depends of how many tokens an embedding model supports**

There's a [HuggingFace dashboard](https://huggingface.co/spaces/mteb/leaderboard) that compares the performacne of different embedding models. Some metrics to focus on are:

- Number of Parameters: 

    A higher value means the model requires more CPU/GPU memory to run

- Embedding Dimension:

    The dimension of the vectors produced

- Max tokens:

    How many tokens the model can process, the higher the better.


For this time, we'll be using the [*all-MiniLM-L6-v2*](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) model:

- Number of Parameters: 22.7M (small)
- Embedding Dimension: 384
- Max tokens: 256

In [24]:
chunk_size = 256
chunk_overlap = 30

text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap = chunk_overlap)

chunks = text_splitter.split_documents(md_header_splits)

chunks

[Document(metadata={'Header 1': 'Resumen Ejecutivo'}, page_content='# Resumen Ejecutivo  \n-----  \n-----'),
 Document(metadata={'Header 1': 'Resumen Ejecutivo', 'Header 2': 'I. Introducción'}, page_content='## I. Introducción  \nLa Reforma Energética es un paso decidido rumbo a la modernización del sector energético de'),
 Document(metadata={'Header 1': 'Resumen Ejecutivo', 'Header 2': 'I. Introducción'}, page_content='nuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamiento de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a'),
 Document(metadata={'Header 1': 'Resumen Ejecutivo', 'Header 2': 'I. Introducción'}, page_content='nivel legistlación secundarias, surge del estudio y valoración de las distintas iniciativas presentadas por los partidos políticos representados en el Congreso.'),
 Document(metadata={'Header 1': 'Resumen Ejecutivo', 'Header 2': 'I. Introducción', 'Header 3': 'La Reforma Energética

## Embeddings



In [32]:
text_to_embed = [doc.page_content for doc in chunks]
text_to_embed

['# Resumen Ejecutivo  \n-----  \n-----',
 '## I. Introducción  \nLa Reforma Energética es un paso decidido rumbo a la modernización del sector energético de',
 'nuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamiento de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a',
 'nivel legistlación secundarias, surge del estudio y valoración de las distintas iniciativas presentadas por los partidos políticos representados en el Congreso.',
 '### La Reforma Energética tiene los siguientes objetivos y premisas fundamentales:  \n1. Mantener la propiedad de la Nación sobre los hidrocarburos que se encuentran en el subsuelo.',
 '2. Modernizar y fortalecer, sin privatizar, a Petróleos Mexicanos (Pemex) y a la Comisión Federal de Electricidad (CFE) como Empresas Productivas del Estado, 100% públicas y 100%\nmexicanas.',
 'mexicanas.\n3. Reducir la exposición del país a los riesgos financieros, geológicos y ambientale

In [33]:
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(f'sentence-transformers/{model_name}')

In [34]:
text_embeddings = model.encode(text_to_embed)

In [35]:
text_embeddings

array([[-0.087618  ,  0.07058339,  0.01356838, ...,  0.03593026,
        -0.0259321 , -0.02630094],
       [-0.00100183,  0.0455025 , -0.0225304 , ...,  0.00107313,
         0.00680774, -0.01094095],
       [ 0.02211484,  0.03129064, -0.10140962, ...,  0.00702693,
         0.0495497 ,  0.04319714],
       ...,
       [-0.00708994,  0.05484241, -0.07352801, ...,  0.00545872,
        -0.00635317,  0.00200859],
       [-0.05489171,  0.03181047,  0.06029795, ...,  0.08888955,
         0.03264892, -0.05778237],
       [-0.00790413,  0.09177164, -0.06890302, ...,  0.06267092,
        -0.02315784, -0.01457029]], shape=(303, 384), dtype=float32)